<a href="https://colab.research.google.com/github/rufous86/hh_api/blob/main/hh_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>

[**hh api**](https://github.com/hhru/api)

</center>

In [91]:
# ! pip install geopy

In [2]:
import requests
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')
import plotly.express as px

In [75]:
res = []
def get_vacancy(vacancies, pages=50):
    for vacancy in tqdm(vacancies):
        print(f'collecting <{vacancy}>')
        for page in tqdm(range(pages)):
            params = {
                'text': f'NAME:{vacancy}',
                'page': page,
                'per_page': 100,
                'currency': 'RUR',
                'only_with_salary': 'true',
                }
            req = requests.get('https://api.hh.ru/vacancies', params).json()
            if 'items' in req.keys():
                res.extend(req['items'])

vacancies = ['maching learning', 'данные', 'data', 'sql',
             'spark', 'hadoop', 'pandas', 'dask', 'deep learning', 'pytorch',
             'tensorflow', 'keras']
get_vacancy(vacancies)

data = pd.DataFrame(res)
data.shape

collecting <maching learning>


  0%|          | 0/50 [00:00<?, ?it/s]

collecting <данные>


  0%|          | 0/50 [00:00<?, ?it/s]

collecting <data>


...

collecting <tensorflow>


  0%|          | 0/50 [00:00<?, ?it/s]

collecting <keras>


  0%|          | 0/50 [00:00<?, ?it/s]

(241, 29)

In [76]:
df = data.copy()
columns = ['name', 'salary', 'schedule', 'area']
df = df[columns]
df.head()

,name,salary,schedule,area
0,Senior Machine Learning Engineer,"{'from': 4000, 'to': 5800, 'currency': 'EUR', ...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '1', 'name': 'Москва', 'url': 'https://..."
1,Machine learning engineer/ разработчик машинно...,"{'from': 200000, 'to': 400000, 'currency': 'RU...","{'id': 'remote', 'name': 'Удаленная работа'}","{'id': '72', 'name': 'Пермь', 'url': 'https://..."
2,Senior Machine Learning Engineer (NLP/Speech),"{'from': 314000, 'to': 314000, 'currency': 'RU...","{'id': 'fullDay', 'name': 'Полный день'}","{'id': '237', 'name': 'Сочи', 'url': 'https://..."
3,Machine Learning Engineer,"{'from': 4000, 'to': 8000, 'currency': 'USD', ...","{'id': 'remote', 'name': 'Удаленная работа'}","{'id': '2758', 'name': 'Тбилиси', 'url': 'http..."
4,Senior Machine Learning Engineer (Golang/Pytho...,"{'from': 380000, 'to': None, 'currency': 'RUR'...","{'id': 'flexible', 'name': 'Гибкий график'}","{'id': '1', 'name': 'Москва', 'url': 'https://..."


In [77]:
df["city"] = (df["area"]
                     .apply(lambda x: x.get("name") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["schedule"] = (df["schedule"]
                     .apply(lambda x: x.get("id") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_from"] = (df["salary"]
                     .apply(lambda x: x.get("from") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_to"] = (df["salary"]
                     .apply(lambda x: x.get("to") 
                                      if isinstance(x, dict) 
                                      else np.nan))
df["salary_currency"] = (df["salary"]
                     .apply(lambda x: x.get("currency") 
                                      if isinstance(x, dict) 
                                      else np.nan))

df = df.drop(['salary', 'area'], axis=1)
df.head()

,name,schedule,city,salary_from,salary_to,salary_currency
0,Senior Machine Learning Engineer,fullDay,Москва,4000.0,5800.0,EUR
1,Machine learning engineer/ разработчик машинно...,remote,Пермь,200000.0,400000.0,RUR
2,Senior Machine Learning Engineer (NLP/Speech),fullDay,Сочи,314000.0,314000.0,RUR
3,Machine Learning Engineer,remote,Тбилиси,4000.0,8000.0,USD
4,Senior Machine Learning Engineer (Golang/Pytho...,flexible,Москва,380000.0,NaN,RUR


In [79]:
df = df[df['salary_currency'] == 'RUR']

In [80]:
df = df.drop_duplicates()

In [81]:
df['salary_currency'].value_counts()

RUR    208
Name: salary_currency, dtype: int64

In [82]:
df['salary_from'] = df['salary_from'].fillna(df['salary_to'])
df['salary_to'] = df['salary_to'].fillna(df['salary_from'])

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208 entries, 1 to 240
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             208 non-null    object 
 1   schedule         208 non-null    object 
 2   city             208 non-null    object 
 3   salary_from      208 non-null    float64
 4   salary_to        208 non-null    float64
 5   salary_currency  208 non-null    object 
dtypes: float64(2), object(4)
memory usage: 11.4+ KB


In [84]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geo")

def get_coords(city):
    geo = geolocator.geocode(city)
    if geo:
        return geo.longitude, geo.latitude
    else:
        return np.nan, np.nan


coords = pd.DataFrame(data=df['city'].unique(), columns=['city'])
coords['(lon, lat)'] = (coords['city']
                        .apply(lambda x: get_coords(x)))


coords['lon'] = (coords['(lon, lat)']
                        .apply(lambda x: x[0]))
coords['lat'] = (coords['(lon, lat)']
                        .apply(lambda x: x[1]))
coords = coords.drop('(lon, lat)', axis=1)

In [85]:
total_vac = dict(df
            .groupby('city')['name']
            .agg('count')
            )

In [86]:
df = df.merge(coords, on='city')
df['salary_mean'] = (df['salary_from'] + df['salary_to']) / 2
df['total_vac'] = (df['city']
                .apply(lambda x: total_vac[x]))
df.head()

,name,schedule,city,salary_from,salary_to,salary_currency,lon,lat,salary_mean,total_vac
0,Machine learning engineer/ разработчик машинно...,remote,Пермь,200000.0,400000.0,RUR,56.230767,58.021487,300000.0,2
1,Разработчик PL/SQL,fullDay,Пермь,80000.0,80000.0,RUR,56.230767,58.021487,80000.0,2
2,Senior Machine Learning Engineer (NLP/Speech),fullDay,Сочи,314000.0,314000.0,RUR,39.723109,43.585482,314000.0,3
3,Middle Data Engineer,fullDay,Сочи,200000.0,200000.0,RUR,39.723109,43.585482,200000.0,3
4,Программист SQL,fullDay,Сочи,35000.0,55000.0,RUR,39.723109,43.585482,45000.0,3


In [87]:
df_grouped = (df
     .groupby('city')[['salary_mean', 'lon', 'lat', 'total_vac']]
     .agg('mean')
                         )

In [88]:
df_grouped['city'] = df_grouped.index

In [89]:
fig = px.scatter_geo(df_grouped, lat='lat', lon='lon', 
                     color='salary_mean', text='city', 
                     color_continuous_scale=px.colors.sequential.Jet, 
                     size='total_vac', size_max=40, width=1000, height=900)
fig.show()